<p>Example script to extract 1 month of hourly MLST MSG product over a domain</p>

In [ ]:
import datetime as dt
import thredds_lsasaf_utils as tlu
import geopandas as gpd
from shapely.geometry import Point

def download_data(dstart, dend, product_freq, LatLonBox):
    # Change here your user credentials
    server_user = "karpagam"
    server_passwd = "chip-chop-2025"

    # Change here the product details
    product_path = "/MSG/MLST/NETCDF/"
    product_fname = "NETCDF4_LSASAF_MSG_LST_MSG-Disk"
    NcvarsLoad = ['LST']  # list of netcdf variables to load from remote files

    # Initialize product details
    product = tlu.lsa_product(product_path, product_fname)
    product.user = server_user
    product.passwd = server_passwd

    # List of slots to be processed:
    slot_list = tlu.gen_slot_list(dstart, dend, product_freq)
    print(f"Will load:{len(slot_list)} files: {slot_list[0]} to {slot_list[-1]}")

    # Load data
    ds_full = tlu.load_product_slots_domain(product, slot_list, NcvarsLoad, LatLonBox=LatLonBox)

    # Extract the temperature DataArray
    temperature_da = ds_full['LST']

    # Stack dimensions (combine 'time', 'lat', and 'lon') and reset the index
    df = temperature_da.stack(points=('time', 'lat', 'lon')).reset_index(['time', 'lat', 'lon']).to_dataframe(name='temperature').reset_index(drop=True)

    # Add additional columns
    df['hour'] = df['time'].dt.hour
    df['day'] = df['time'].dt.day
    df['month'] = df['time'].dt.month
    df['year'] = df['time'].dt.year

    # Create geometry from latitude and longitude
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Set the Coordinate Reference System (CRS) - assuming WGS84 (EPSG:4326)
    gdf.set_crs(epsg=4326, inplace=True)

    return gdf


# Time period to process
dstart = dt.datetime(2024, 7, 3, 0, 0, 0)  # start slot
dend = dt.datetime(2024, 7, 31, 23, 59, 59)  # end slot

# Frequency
product_freq = "h"  # hourly frequency

# Define latitude/longitude domain to load [lat_min, lat_max, lon_min, lon_max]
LatLonBox = [41.7196, 42.1012, 12.2406, 12.7503]  # Rome

# Loop through each day
current_day = dstart
while current_day <= dend:
    next_day = current_day + dt.timedelta(days=1) - dt.timedelta(seconds=1)

    # Download data for the current day
    gdf = download_data(current_day, next_day, product_freq, LatLonBox)


    # Create filename for GeoPackage
    gpkg_filename = f'../downloads/MSG_2ND/LST_{current_day.year}-{current_day.month:02d}-{current_day.day:02d}.gpkg'

    # Save to GeoPackage
    gdf.to_file(gpkg_filename, driver="GPKG")

    print(f'GeoDataFrame saved for {current_day.date()} as {gpkg_filename}')

    # Move to the next day
    current_day = next_day + dt.timedelta(seconds=1)